In [ ]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

from sklearn.preprocessing import scale
from sklearn.model_selection import train_test_split

from sklearn import metrics
from sklearn.metrics import confusion_matrix
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV, StratifiedKFold

from sklearn.decomposition import PCA

import joblib

In [ ]:
#import file and reading few lines
# dataTrain = pd.read_csv('FeaturesWhite.csv')
# dataEval = pd.read_csv('EvalWithWhite.csv')
# dataTrain = pd.read_csv('FeaturesNoWhite.csv')
# dataEval = pd.read_csv('EvalWithNoWhite.csv')
# dataTrain = pd.read_csv('HistogramWhite.csv')
# dataEval = pd.read_csv('EvalHistogramWhite.csv')
dataTrain = pd.read_csv('ProfileWhite.csv')
dataEval = pd.read_csv('EvalProfileWhite.csv')
#dataTrain.head(10)
file = open('Accuracies.txt', 'a')
listOfLines=[]

In [ ]:
print(dataTrain.shape)
print(dataEval.shape)

(2728, 824)
(682, 824)


In [ ]:
cnames = list(dataTrain.columns)
print(cnames)

['image', 'label', 'BlackToWhite', 'Aspect Ratio', 'Column Histogram 0', 'Column Histogram 1', 'Column Histogram 2', 'Column Histogram 3', 'Column Histogram 4', 'Column Histogram 5', 'Column Histogram 6', 'Column Histogram 7', 'Column Histogram 8', 'Column Histogram 9', 'Column Histogram 10', 'Column Histogram 11', 'Column Histogram 12', 'Column Histogram 13', 'Column Histogram 14', 'Column Histogram 15', 'Column Histogram 16', 'Column Histogram 17', 'Column Histogram 18', 'Column Histogram 19', 'Column Histogram 20', 'Column Histogram 21', 'Column Histogram 22', 'Column Histogram 23', 'Column Histogram 24', 'Column Histogram 25', 'Column Histogram 26', 'Column Histogram 27', 'Column Histogram 28', 'Column Histogram 29', 'Row Histogram 0', 'Row Histogram 1', 'Row Histogram 2', 'Row Histogram 3', 'Row Histogram 4', 'Row Histogram 5', 'Row Histogram 6', 'Row Histogram 7', 'Row Histogram 8', 'Row Histogram 9', 'Row Histogram 10', 'Row Histogram 11', 'Row Histogram 12', 'Row Histogram 13',

In [ ]:
ndataTrain = dataTrain
ndataEval = dataEval

cnames = list(dataTrain.columns)

#remove HOG
# for c in cnames:
#   if c.find('HOG') != -1:
#     ndataTrain = ndataTrain.drop(c,axis =1)
#     ndataEval = ndataEval.drop(c,axis =1)


#remove Profile
# for c in cnames:
#   if c.find('Top') != -1 or c.find('Right') != -1 or c.find('Left') != -1 or c.find('Top') != -1 or c.find('Bottom') != -1:
#     ndataTrain = ndataTrain.drop(c,axis =1)
#     ndataEval = ndataEval.drop(c,axis =1)


# remove Histogram
# for c in cnames:
#   if c.find('Histogram') != -1:
#     ndataTrain = ndataTrain.drop(c,axis =1)
#     ndataEval = ndataEval.drop(c,axis =1)


# remove Symmetry
# for c in cnames:
#   if c.find('Symmetry') != -1:
#     ndataTrain = ndataTrain.drop(c,axis =1)
#     ndataEval = ndataEval.drop(c,axis =1)


In [ ]:
ndataTrain.describe(percentiles = [0.05,0.10,0.25,0.50,0.75,0.90,0.99])

,BlackToWhite,Aspect Ratio,Column Histogram 0,Column Histogram 1,Column Histogram 2,Column Histogram 3,Column Histogram 4,Column Histogram 5,Column Histogram 6,Column Histogram 7,...,HOG 630,HOG 631,HOG 632,HOG 633,HOG 634,HOG 635,HOG 636,HOG 637,HOG 638,HOG 639
count,2728.000000,2728.000000,2728.000000,2728.000000,2728.000000,2728.000000,2728.000000,2728.000000,2728.000000,2728.000000,...,2728.000000,2728.000000,2728.000000,2728.000000,2728.000000,2728.000000,2728.000000,2728.000000,2728.000000,2728.000000
mean,0.920930,0.793757,0.032806,0.075754,0.123485,0.187879,0.262204,0.337069,0.405489,0.451683,...,0.028547,0.002413,0.023749,0.009320,0.118547,0.027727,0.116288,0.015298,0.028397,0.002088
std,0.574493,0.256534,0.079144,0.150864,0.201066,0.245763,0.282415,0.295835,0.295638,0.281953,...,0.065096,0.010147,0.056216,0.024494,0.103208,0.033364,0.122328,0.035007,0.055567,0.008343
min,0.250764,0.145482,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
5%,0.416202,0.444991,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
10%,0.474304,0.525757,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.100000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.613739,0.636770,0.000000,0.000000,0.000000,0.000000,0.000000,0.050654,0.166667,0.233333,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.789889,0.768109,0.000000,0.000000,0.000000,0.039412,0.166667,0.279020,0.366667,0.421111,...,0.000000,0.000000,0.000000,0.000000,0.117437,0.016410,0.076482,0.000000,0.000000,0.000000
75%,1.073029,0.919418,0.000000,0.100000,0.200000,0.333333,0.466667,0.566667,0.633333,0.666667,...,0.000000,0.000000,0.011913,0.000000,0.218082,0.045562,0.239293,0.000000,0.036160,0.000000
90%,1.423110,1.106355,0.133333,0.300000,0.455556,0.566667,0.700000,0.766667,0.860235,0.895961,...,0.126002,0.000000,0.078463,0.033085,0.257358,0.072007,0.284325,0.064247,0.103458,0.000000


In [ ]:
np.unique(ndataTrain['label'])

array(['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', 'A', 'B', 'C',
       'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P',
       'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', 'a', 'b', 'c',
       'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p',
       'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z'], dtype=object)

In [ ]:
ndataTrain['label'].value_counts()

0    44
k    44
X    44
Y    44
Z    44
     ..
P    44
Q    44
R    44
S    44
z    44
Name: label, Length: 62, dtype: int64

In [ ]:
y_train = ndataTrain['label']
y_eval = ndataEval['label']
ndataTrain = ndataTrain.drop("label", axis = 1)
ndataTrain = ndataTrain.drop("image", axis = 1)
ndataEval = ndataEval.drop("label", axis = 1)
ndataEval = ndataEval.drop("image", axis = 1)

In [ ]:
X_train = ndataTrain
X_eval = ndataEval
# train test split
# X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.7,test_size = 0.3, random_state = 101)
# pca = PCA(n_components=50)
# X_train = pca.fit_transform(X_train)
# X_test = pca.transform(X_test)

In [ ]:
print('X_train shape:',X_train.shape)
print('y_train shape:',y_train.shape)
print('X_test shape:',X_eval.shape)
print('y_test shape:',y_eval.shape)

X_train shape: (2728, 822)
y_train shape: (2728,)
X_test shape: (682, 822)
y_test shape: (682,)


MODEL BUILDING

In [ ]:
v = X_train.isnull().any()
#print(v)
print(v[v == True])

Series([], dtype: bool)


In [ ]:
model_linear = SVC(kernel='linear', probability = True)
model_linear.fit(X_train, y_train)

# predict
y_pred = model_linear.predict_proba(X_eval)

In [ ]:
y_pred = model_linear.predict_proba(X_eval)
print(str(len(y_pred[0])))

62


In [ ]:

predictions_proba = y_pred[10]
def Top3(predictions_proba):

  predictions_proba = predictions_proba.tolist()
  lea = "0123456789ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz"
  predictions=[]

  index1 = predictions_proba.index(max(predictions_proba))
  predictions.append(lea[index1])
  predictions.append(predictions_proba[index1]) 
  predictions_proba[index1] = 0

  index2 = predictions_proba.index(max(predictions_proba))
  predictions.append(lea[index2])
  predictions.append(predictions_proba[index2]) 
  predictions_proba[index2] = 0

  index3 = predictions_proba.index(max(predictions_proba))
  predictions.append(lea[index3])
  predictions.append(predictions_proba[index3]) 
  predictions_proba[index3] = 0
 
  return predictions_proba, predictions

predictions_proba, predictions= Top3(predictions_proba)
print(len(predictions_proba))
print(predictions)

AttributeError: ignored

In [ ]:
# accuracy
print("accuracy:", metrics.accuracy_score(y_true=y_eval, y_pred=y_pred), "\n")

accuracy: 0.8299120234604106 



In [ ]:
print(np.unique(y_train))
print(y_train.value_counts())
print(y_eval.value_counts())

['0' '1' '2' '3' '4' '5' '6' '7' '8' '9' 'A' 'B' 'C' 'D' 'E' 'F' 'G' 'H'
 'I' 'J' 'K' 'L' 'M' 'N' 'O' 'P' 'Q' 'R' 'S' 'T' 'U' 'V' 'W' 'X' 'Y' 'Z'
 'a' 'b' 'c' 'd' 'e' 'f' 'g' 'h' 'i' 'j' 'k' 'l' 'm' 'n' 'o' 'p' 'q' 'r'
 's' 't' 'u' 'v' 'w' 'x' 'y' 'z']
0    44
k    44
X    44
Y    44
Z    44
     ..
P    44
Q    44
R    44
S    44
z    44
Name: label, Length: 62, dtype: int64
0    11
k    11
X    11
Y    11
Z    11
     ..
P    11
Q    11
R    11
S    11
z    11
Name: label, Length: 62, dtype: int64


In [ ]:
# cm
print(metrics.confusion_matrix(y_true=y_eval, y_pred=y_pred))

[[ 4  0  0 ...  0  0  0]
 [ 0  9  0 ...  0  0  0]
 [ 0  0 11 ...  0  0  0]
 ...
 [ 0  0  0 ...  8  0  0]
 [ 0  0  0 ...  0  9  0]
 [ 0  0  1 ...  0  0  6]]


In [ ]:
#precision, recall and f1-score
scores=metrics.classification_report(y_eval, y_pred)
print(scores)

              precision    recall  f1-score   support

           0       0.67      0.36      0.47        11
           1       0.56      0.82      0.67        11
           2       0.92      1.00      0.96        11
           3       0.92      1.00      0.96        11
           4       0.92      1.00      0.96        11
           5       0.91      0.91      0.91        11
           6       1.00      1.00      1.00        11
           7       1.00      1.00      1.00        11
           8       0.91      0.91      0.91        11
           9       0.77      0.91      0.83        11
           A       1.00      1.00      1.00        11
           B       0.85      1.00      0.92        11
           C       0.83      0.91      0.87        11
           D       1.00      1.00      1.00        11
           E       1.00      1.00      1.00        11
           F       1.00      1.00      1.00        11
           G       1.00      1.00      1.00        11
           H       0.85    

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


MODEL 2

In [ ]:
# non-linear model
# using poly kernel, C=1, default value of gamma

# model
non_linear_model_poly = SVC(kernel='poly')

# fit
non_linear_model_poly.fit(X_train, y_train)

# predict
y_pred = non_linear_model_poly.predict(X_eval)

In [ ]:
# confusion matrix and accuracy, precision, recall

# accuracy
print("accuracy:", metrics.accuracy_score(y_true=y_eval, y_pred=y_pred), "\n")

# cm
print(metrics.confusion_matrix(y_true=y_eval, y_pred=y_pred))

accuracy: 0.8181818181818182 

[[ 4  0  0 ...  0  0  0]
 [ 0  6  0 ...  0  0  0]
 [ 0  0 11 ...  0  0  0]
 ...
 [ 0  0  0 ...  9  0  0]
 [ 0  0  0 ...  0  7  0]
 [ 0  0  1 ...  0  0  6]]


MODEL 3

In [ ]:
# non-linear model
# using rbf kernel, C=1, default value of gamma

# model
non_linear_model = SVC(kernel='rbf')

# fit
non_linear_model.fit(X_train, y_train)

# predict
y_pred = non_linear_model.predict(X_eval)

In [ ]:
# confusion matrix and accuracy, precision, recall
# accuracy
print("accuracy:", metrics.accuracy_score(y_true=y_eval, y_pred=y_pred), "\n")

# cm
print(metrics.confusion_matrix(y_true=y_eval, y_pred=y_pred))

accuracy: 0.8167155425219942 

[[ 2  0  0 ...  0  0  0]
 [ 0  8  0 ...  0  0  0]
 [ 0  0 11 ...  0  0  0]
 ...
 [ 0  0  0 ...  9  0  0]
 [ 0  0  0 ...  0  7  0]
 [ 0  1  2 ...  0  0  6]]


In [ ]:
#precision, recall and f1-score
scores=metrics.classification_report(y_eval, y_pred)
print(scores)

              precision    recall  f1-score   support

           0       0.50      0.18      0.27        11
           1       0.42      0.73      0.53        11
           2       0.79      1.00      0.88        11
           3       0.83      0.91      0.87        11
           4       0.79      1.00      0.88        11
           5       0.89      0.73      0.80        11
           6       0.92      1.00      0.96        11
           7       1.00      0.91      0.95        11
           8       0.83      0.91      0.87        11
           9       0.64      0.82      0.72        11
           A       0.92      1.00      0.96        11
           B       1.00      1.00      1.00        11
           C       0.71      0.91      0.80        11
           D       1.00      1.00      1.00        11
           E       1.00      1.00      1.00        11
           F       1.00      1.00      1.00        11
           G       1.00      0.91      0.95        11
           H       1.00    

Grid Search: Hyperparameter Tuning

In [ ]:
#SVM
hyper_param = {'C': [0.1, 1, 10, 100], 
               'gamma': [1, 0.1, 0.01, 0.001], 
               'kernel': ['linear', 'rbf', 'poly']}
model = SVC(probability = True)
gs = GridSearchCV(estimator=model,
                    param_grid=hyper_param,
                    cv=StratifiedKFold(5).split(X_train, y_train),
                    scoring='accuracy',
                    refit=True,
                    verbose=3)

gs.fit(X_train, y_train)

# save the model to disk
joblib.dump(gs.best_estimator_, 'FinalSVCProfileWhite.npy')


Fitting 5 folds for each of 48 candidates, totalling 240 fits
[CV 1/5] END .....C=0.1, gamma=1, kernel=linear;, score=0.799 total time=   5.1s
[CV 2/5] END .....C=0.1, gamma=1, kernel=linear;, score=0.853 total time=   6.4s
[CV 3/5] END .....C=0.1, gamma=1, kernel=linear;, score=0.835 total time=   5.1s
[CV 4/5] END .....C=0.1, gamma=1, kernel=linear;, score=0.828 total time=   5.1s
[CV 5/5] END .....C=0.1, gamma=1, kernel=linear;, score=0.771 total time=   4.9s
[CV 1/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.084 total time=  16.6s
[CV 2/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.084 total time=  17.1s
[CV 3/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.088 total time=  21.1s
[CV 4/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.119 total time=  15.2s
[CV 5/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.094 total time=  16.1s
[CV 1/5] END .......C=0.1, gamma=1, kernel=poly;, score=0.793 total time=   4.9s
[CV 2/5] END .......C=0.1, gamma=1, kernel=poly

['FinalSVCProfileWhite.npy']

In [ ]:
l = 'Best Params of SVC with Profile White: '+str(gs.best_params_)+'\n'
l2 = 'Best Accuracy of SVC with Profile White: '+str(gs.best_score_)+'\n'
print(l)
print(l2)
y_pred = gs.predict(X_eval)

# metrics
print("EVAL accuracy", metrics.accuracy_score(y_eval, y_pred), "\n")
print(metrics.confusion_matrix(y_eval, y_pred), "\n")



Best Params of SVC with Profile White: {'C': 100, 'gamma': 0.001, 'kernel': 'rbf'}

Best Accuracy of SVC with Profile White: 0.8295312027422119

EVAL accuracy 0.8284457478005866 

[[ 4  0  0 ...  0  0  0]
 [ 0  9  0 ...  0  0  0]
 [ 0  0 11 ...  0  0  0]
 ...
 [ 0  0  0 ...  9  0  0]
 [ 0  0  0 ...  0  9  0]
 [ 0  1  1 ...  0  0  6]] 



In [ ]:
#KNN
from sklearn import neighbors
knn = neighbors.KNeighborsClassifier()
k_range = list(range(1, 31))
param_grid = dict(n_neighbors=k_range)
  
# defining parameter range
gsKNN = GridSearchCV(knn, 
                    param_grid, 
                    cv=StratifiedKFold(5).split(X_train, y_train), 
                    scoring='accuracy',
                    verbose=3)
  
# fitting the model for grid search
grid_search=gsKNN.fit(X_train, y_train)

# save the model to disk
joblib.dump(gsKNN.best_estimator_, 'BestKNNProfileWhite.npy')

Fitting 5 folds for each of 30 candidates, totalling 150 fits
[CV 1/5] END .....................n_neighbors=1;, score=0.698 total time=   0.1s
[CV 2/5] END .....................n_neighbors=1;, score=0.808 total time=   0.1s
[CV 3/5] END .....................n_neighbors=1;, score=0.775 total time=   0.1s
[CV 4/5] END .....................n_neighbors=1;, score=0.767 total time=   0.2s
[CV 5/5] END .....................n_neighbors=1;, score=0.701 total time=   0.2s
[CV 1/5] END .....................n_neighbors=2;, score=0.698 total time=   0.2s
[CV 2/5] END .....................n_neighbors=2;, score=0.771 total time=   0.1s
[CV 3/5] END .....................n_neighbors=2;, score=0.753 total time=   0.1s
[CV 4/5] END .....................n_neighbors=2;, score=0.730 total time=   0.1s
[CV 5/5] END .....................n_neighbors=2;, score=0.651 total time=   0.2s
[CV 1/5] END .....................n_neighbors=3;, score=0.718 total time=   0.2s
[CV 2/5] END .....................n_neighbors=3

['BestKNNProfileWhite.npy']

In [ ]:
l = 'Best Params of KNN with Profile White: '+str(gsKNN.best_params_)+'\n'
l2 = 'Best Accuracy of KNN with Profile White: '+str(gsKNN.best_score_)+'\n'
print(l)
print(l2)

Best Params of KNN with Profile White: {'n_neighbors': 1}

Best Accuracy of KNN with Profile White: 0.7496219376953321



In [ ]:
#RandomForest
from sklearn.ensemble import RandomForestClassifier
rfc=RandomForestClassifier(random_state=42, probability = True)
param_grid = { 
    'n_estimators': [50, 200],
    'max_depth' : [4,5,6,7,8],
    'criterion' :['entropy']
}
  
# defining parameter range
gsRFC = GridSearchCV(estimator=rfc, 
                     param_grid=param_grid, 
                     cv=StratifiedKFold(5).split(X_train, y_train), 
                    scoring='accuracy',
                    verbose=3)
gsRFC.fit(X_train, y_train)
  
# save the model to disk
joblib.dump(gsRFC.best_estimator_, 'FinalRFCProfileWhite.npy')

TypeError: ignored

In [ ]:
l = 'Best Params of RFC with profile White: '+str(gsRFC.best_params_)+'\n'
l2 = 'Best Accuracy of RFC with profile White: '+str(gsRFC.best_score_)+'\n'
print(l)
print(l2)

In [ ]:
# cv results
cv_results = pd.DataFrame(gs.cv_results_)
cv_results
print(gsKNN.best_estimator_)

KNeighborsClassifier(n_neighbors=1)


In [ ]:
# printing the optimal accuracy score and hyperparameters
best_score = gs.best_score_
best_hyperparams = gs.best_params_

print("The best test score of SVMis {0} corresponding to hyperparameters {1}".format(best_score, best_hyperparams))

best_scoreKNN = gsKNN.best_score_
best_hyperparamsKNN = gsKNN.best_params_

print("The best test score of KNNis {0} corresponding to hyperparameters {1}".format(best_scoreKNN, best_hyperparamsKNN))

best_scoreRFC = gsRFC.best_score_
best_hyperparamsRFC = gsRFC.best_params_

print("The best test score of KNNis {0} corresponding to hyperparameters {1}".format(best_scoreRFC, best_hyperparamsRFC))

The best test score of SVMis 0.8295312027422119 corresponding to hyperparameters {'C': 100, 'gamma': 0.001, 'kernel': 'rbf'}
The best test score of KNNis 0.7496219376953321 corresponding to hyperparameters {'n_neighbors': 1}
The best test score of KNNis 0.8078986456968108 corresponding to hyperparameters {'criterion': 'entropy', 'max_depth': 8, 'n_estimators': 200}


Building and Evaluating the Final Model

In [ ]:
# model with optimal hyperparameters

# model SVC
model = joblib.load('BestSVCProfileWhite.npy')

y_pred = model.predict(X_eval)

# metrics
print("EVAL accuracy", metrics.accuracy_score(y_eval, y_pred), "\n")
print(metrics.confusion_matrix(y_eval, y_pred), "\n")


#MODEL KNN
modelKNN = joblib.load('BestKNNProfileWhite.npy')

y_pred = modelKNN.predict(X_eval)

# metrics
print("EVAL accuracy", metrics.accuracy_score(y_eval, y_pred), "\n")
print(metrics.confusion_matrix(y_eval, y_pred), "\n")


#MODEL RFC
modelRFC = joblib.load('BestRFCProfileWhite.npy')

y_pred = modelRFC.predict(X_eval)

# metrics
print("EVAL accuracy", metrics.accuracy_score(y_eval, y_pred), "\n")
print(metrics.confusion_matrix(y_eval, y_pred), "\n")

EVAL accuracy 0.8284457478005866 

[[ 4  0  0 ...  0  0  0]
 [ 0  9  0 ...  0  0  0]
 [ 0  0 11 ...  0  0  0]
 ...
 [ 0  0  0 ...  9  0  0]
 [ 0  0  0 ...  0  9  0]
 [ 0  1  1 ...  0  0  6]] 

EVAL accuracy 0.7859237536656891 

[[6 0 0 ... 0 0 0]
 [0 6 1 ... 0 0 0]
 [0 0 8 ... 0 0 1]
 ...
 [0 0 0 ... 8 0 0]
 [0 0 0 ... 0 5 0]
 [0 1 1 ... 0 0 7]] 

EVAL accuracy 0.8108504398826979 

[[2 0 0 ... 0 0 0]
 [0 9 1 ... 0 0 0]
 [0 0 8 ... 0 0 0]
 ...
 [0 0 0 ... 8 0 0]
 [0 0 0 ... 0 7 0]
 [0 0 1 ... 0 0 5]] 



In [ ]:
###Parralel Ensemble
def find_top_3(predictions_proba):

  characters = "0123456789ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz"
  predictions=[]

  predictions_proba = [round(member, 2) for member in predictions_proba]

  max_number = max(predictions_proba)
  index1 = predictions_proba.index(max_number)
  predictions.append(characters[index1])
  predictions.append(predictions_proba[index1]) 
  predictions_proba[index1] = 0

  index2 = predictions_proba.index(max(predictions_proba))
  predictions.append(characters[index2])
  predictions.append(predictions_proba[index2]) 
  predictions_proba[index2] = 0

  index3 = predictions_proba.index(max(predictions_proba))
  predictions.append(characters[index3])
  predictions.append(predictions_proba[index3]) 
  predictions_proba[index3] = 0
 
  return predictions_proba, predictions

def Parallel_Ensemb(input, model1, model2, model3, w1, w2, w3):

  y_model1_proba = model1.predict_proba(input)
  y_model1_proba = y_model1_proba.tolist()
  y_model1_proba = [value * w1 for value in y_model1_proba]

  y_model2_proba = model2.predict_proba(input)
  y_model2_proba = y_model2_proba.tolist()
  y_model2_proba = [value * w2 for value in y_model2_proba]

  y_model3_proba = model3.predict_proba(input)
  y_model3_proba = y_model3_proba.tolist()
  y_model3_proba = [value * w3 for value in y_model3_proba]

  final_y_proba = sum = [x + y for (x, y) in zip(y_model1_proba, y_model2_proba)] 
  final_y_proba = sum = [x + y for (x, y) in zip(final_y_proba, y_model3_proba)] 
  
  _, predictions = find_top_3(final_y_proba)

  return predictions


In [ ]:
import joblib

model1 = joblib.load('FinalSVCProfileWhite.npy')
model2 = joblib.load('BestKNNHISTWhite.npy')
model3 = joblib.load('BestRFCProfileWhite.npy')

y_pred1 = model1.predict_proba(X_eval).tolist()
y_pred2 = model2.predict_proba(X_eval).tolist()
y_pred3 = model3.predict_proba(X_eval).tolist()
print(len(y_pred1[57]))
print(len(y_pred2[57]))
print(len(y_pred3[57]))

# y_pred1 = ["%.2f" % member for member in y_pred1[0]]
# y_pred2 = ["%.2f" % member for member in y_pred2[0]]
# y_pred3 = ["%.2f" % member for member in y_pred3[0]]
# print(y_pred1)
y_pred_proba1 = [value * 0.3 for value in y_pred1[57]]
y_pred_proba2 = [value * 0.4 for value in y_pred[57]]
y_pred_proba3 = [value * 0.3 for value in y_pred3[57]]

y_pred_proba = sum = [x + y for (x, y) in zip(y_pred_proba1, y_pred_proba2)] 
y_pred_proba = sum = [x + y for (x, y) in zip(y_pred_proba, y_pred_proba3)] 
print(len(y_pred_proba))
# print(y_pred_proba)
_, predictions = find_top_3(y_pred_proba)
print(predictions)
# y_pred3 = ["%.2f" % member for member in y_pred3[10]]



62
62
62
62
[0.01, 0.0, 0.0, 0.02, 0.0, 0.66, 0.02, 0.0, 0.01, 0.01, 0.0, 0.01, 0.0, 0.0, 0.04, 0.01, 0.02, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.06, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.01, 0.0, 0.01, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.04, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
0.66
5
5
['5', 0.66, 'S', 0.06, 'E', 0.04]
